<a href="https://colab.research.google.com/github/torrhen/pytorch/blob/main/pytorch_experiment_tracking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# import torch and torchvision libraries
try:
  import torch
  import torchvision
  assert(torch.__version__.split(".")[1] >= 12)
  assert(torchvision.__version__.split(".")[1] >= 13)
  print(f"torch version: {torch.__version__}")
  print(f"torchvision version: {torchvision.__version__}")
# install torch and torchvision libraries if necessary
except:
  !pip3 install -U torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu113
  import torch
  import torchvision
  print(f"torch version: {torch.__version__}")
  print(f"torchvision version: {torchvision.__version__}")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/, https://download.pytorch.org/whl/cu113
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸ 887.4/887.4 MB 54.2 MB/s eta 0:00:01tcmalloc: large alloc 1109270528 bytes == 0x377dc000 @  0x7f5756233615 0x5d6f4c 0x51edd1 0x51ef5b 0x4f750a 0x4997a2 0x55cd91 0x5d8941 0x4997a2 0x55cd91 0x5d8941 0x4997a2 0x55cd91 0x5d8941 0x4997a2 0x55cd91 0x5d8941 0x4997a2 0x55cd91 0x5d8941 0x4997a2 0x5d8868 0x4997a2 0x55cd91 0x5d8941 0x49abe4 0x55cd91 0x5d8941 0x4997a2 0x55cd91 0x5d8941
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.4/887.4 MB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.2/24.2 MB 51.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 66.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 59.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 KB 37.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import matplotlib.pyplot as plt
import torch
import torchvision
from torch import nn
from torchvision import transforms

# import torchinfo summary
try:
  from torchinfo import summary
# install torchinfo summary if necessary
except:
  !pip install -q torchinfo
  from torchinfo import summary

# import third party scripts
try:
  from going_modular.going_modular import data_setup, engine
# clone github scripts if necessary
except:
  !git clone https://github.com/mrdbourke/pytorch-deep-learning
  !mv pytorch-deep-learning/going_modular .
  !rm -rf pytorch-deep-learning
  from going_modular.going_modular import data_setup, engine

Cloning into 'pytorch-deep-learning'...
remote: Enumerating objects: 3369, done.
remote: Counting objects: 100% (67/67), done.
remote: Compressing objects: 100% (43/43), done.
remote: Total 3369 (delta 25), reused 49 (delta 23), pack-reused 3302
Receiving objects: 100% (3369/3369), 641.24 MiB | 31.44 MiB/s, done.
Resolving deltas: 100% (1932/1932), done.
Checking out files: 100% (221/221), done.


In [3]:
# device agnostic code
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [4]:
# set the random seed
def set_seed(seed: int = 42):
  torch.manual_seed(seed)
  torch.cuda.manual_seed(seed)

In [5]:
import os
import zipfile
from pathlib import Path

import requests

# create folder paths
data_path = Path("data/")
image_path = data_path / "pizza_steak_sushi"

# check if images folder already exists
if image_path.is_dir():
  print(f"{image_path} already exists")
else:
  # create image folder
  image_path.mkdir(parents=True, exist_ok=True)
  # download the training and test image zip folder from GitHub
  with open(data_path / "pizza_steak_sushi.zip", "wb") as f:
    response = requests.get("https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip")
    f.write(response.content)
  # extract images zip folder
  with zipfile.ZipFile(data_path / "pizza_steak_sushi.zip", "r") as z:
    z.extractall(image_path)
  # remove images zip folder after extraction
  os.remove(data_path / "pizza_steak_sushi.zip")

In [6]:
# store the file paths of the training and test images
train_folder = image_path / "train"
test_folder = image_path / "test"

In [10]:
# store the weights of the pretrained model
weights = torchvision.models.EfficientNet_B0_Weights.DEFAULT
# store the transforms used to obtain the weights
auto_transform = weights.transforms()

# divide the training and test data into batches
train_loader, test_loader, class_names = data_setup.create_dataloaders(train_dir=train_folder,
                                                                   test_dir = test_folder,
                                                                   transform=auto_transform,
                                                                   batch_size=32)

print(len(train_loader))
print(len(test_loader))

8
3


In [12]:
# create model using pretrained weights
model_0 = torchvision.models.efficientnet_b0(weights=weights).to(device)

# freeze the weights of the features component of the model
for param in model_0.features.parameters():
  param.requires_grad = False

# set random seed
set_seed()

# modify the classifier of the model to accomodate the classes of data
model_0.classifier = nn.Sequential(
    nn.Dropout(p=0.2, inplace=True),
    nn.Linear(in_features=1280, out_features=len(class_names), bias=True)
).to(device)

In [13]:
from torchinfo import summary

# display architecture of the model
summary(model_0, input_size=(32, 3, 224, 244), col_names=['input_size', 'output_size', 'trainable'])

Layer (type:depth-idx)                                  Input Shape               Output Shape              Trainable
EfficientNet                                            [32, 3, 224, 244]         [32, 3]                   Partial
├─Sequential: 1-1                                       [32, 3, 224, 244]         [32, 1280, 7, 8]          False
│    └─Conv2dNormActivation: 2-1                        [32, 3, 224, 244]         [32, 32, 112, 122]        False
│    │    └─Conv2d: 3-1                                 [32, 3, 224, 244]         [32, 32, 112, 122]        False
│    │    └─BatchNorm2d: 3-2                            [32, 32, 112, 122]        [32, 32, 112, 122]        False
│    │    └─SiLU: 3-3                                   [32, 32, 112, 122]        [32, 32, 112, 122]        --
│    └─Sequential: 2-2                                  [32, 32, 112, 122]        [32, 16, 112, 122]        False
│    │    └─MBConv: 3-4                                 [32, 32, 112, 122]        [32